In [2]:
pip install --upgrade nbformat

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\SUBBU\Downloads\TransferIQ-Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\SUBBU\Downloads\TransferIQ-Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import plotly.graph_objects as go

# --- 1. Load and Prepare the Final Dataset ---
try:
    df_perf = pd.read_csv('Full Dataset (league standings with players stats).csv')
    df_perf = df_perf.loc[:, ~df_perf.columns.str.contains('^Unnamed')]
    df_perf['fullname'] = df_perf['firstname'] + ' ' + df_perf['lastname']
    player_name = "James Philip Milner"
    df_player = df_perf[df_perf['fullname'] == player_name].copy()
    df_player = df_player.sort_values('season').dropna(subset=['totalGoals', 'goalAssists', 'totalShots'])
    print(f"✅ Data prepared for {player_name}")
except FileNotFoundError:
    print("❌ File not found.")
    exit()

# --- 2. Create the Full Time-Series ---
features = ['totalGoals', 'goalAssists', 'totalShots']
target = 'totalGoals'
lookback = 2
X, y = [], []
for i in range(lookback, len(df_player)):
    X.append(df_player[features].iloc[i-lookback:i].values)
    y.append(df_player[target].iloc[i])

X = np.array(X)
y = np.array(y)

# --- 3. Train the Champion Model (LSTM) on ALL Data ---
print("\n--- Training final LSTM model on all available data... ---")
scaler_X = MinMaxScaler(feature_range=(0, 1)); scaler_y = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X.reshape(-1, X.shape[2])).reshape(X.shape)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

final_lstm_model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    LSTM(units=50),
    Dense(units=1)
])
final_lstm_model.compile(optimizer='adam', loss='mean_squared_error')
final_lstm_model.fit(X_scaled, y_scaled, epochs=100, batch_size=1, verbose=0)
print("✅ Final model trained.")

# --- 4. Generate Predictions to Visualize ---
# We will "predict" on the training data to see how well the model learned the patterns
predictions_scaled = final_lstm_model.predict(X_scaled)
predictions = scaler_y.inverse_transform(predictions_scaled)

# --- 5. Create the Interactive Plot with Plotly ---
# Get the seasons for the x-axis label
seasons = df_player['season'].iloc[lookback:].tolist()

fig = go.Figure()

# Add the 'Actual Goals' line
fig.add_trace(go.Scatter(
    x=seasons, 
    y=y.flatten(),
    mode='lines+markers',
    name='Actual Goals',
    line=dict(color='blue', width=3)
))

# Add the 'Predicted Goals' line
fig.add_trace(go.Scatter(
    x=seasons, 
    y=predictions.flatten(),
    mode='lines+markers',
    name='Predicted Goals',
    line=dict(color='red', dash='dash')
))

fig.update_layout(
    title=f"Actual vs. Predicted Goals for {player_name}",
    xaxis_title="Season",
    yaxis_title="Total Goals",
    legend_title="Legend",
    font=dict(family="Arial, sans-serif", size=14)
)

fig.show()

✅ Data prepared for James Philip Milner

--- Training final LSTM model on all available data... ---


c:\Users\SUBBU\Downloads\TransferIQ-Project\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Final model trained.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
